## Importing Essential Libraries

In [ ]:
from google.adk.agents import LlmAgent, Agent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory, google_search, AgentTool
from google.genai import types
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional

print("✅ ADK components imported successfully.")

## Setup Gemini Api Key

In [ ]:
try:
    GOOGLE_API_KEY = load_dotenv("GOOGLE_API_KEY")
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

## Configure Retry Options

In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

## Creating the Agent Instance

In [ ]:
#!adk create mediflow_ai --model gemini-2.0-flash --api_key $GOOGLE_API_KEY

# Creating the run_session async function to run Queries

In [ ]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

# Creating Callbacks to Auto save to Memory

In [ ]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("✅ Callback created.")

# Initializing APP NAME and USER ID

In [ ]:
APP_NAME = "MediFlow_AI"
USER_ID = "bidit_test"

## Initializing Session and Memory Service


In [ ]:
memory_service = (
    InMemoryMemoryService()
) 
session_service = InMemorySessionService() 

# Defining Agents

### Google Search Agent

In [ ]:

google_search_agent = LlmAgent(
    name="GoogleSearchAgent",
    model=Gemini(
        model_name="gemini-2.0-flash",
        retry_options=retry_config,
    ),
    description='''
You are GoogleSearchAgent, an expert in finding accurate and relevant information using Google Search.
Your task is to assist the main agent by performing Google searches based on user queries and returning concise summaries of the search results. ''',
    instruction = '''
You will be provided with a user's query. Use Google Search to find the most relevant and up-to-date information. It should be fact based and objective.
Store the search results in 'google_search_output'.
''',
    tools=[google_search],
    output_key="google_search_output"
)

### Triage Doctor Finder Agent

In [ ]:
triage_doctor_finder_agent = LlmAgent(
    name="triage_doctor_finder_agent",
    model = "gemini-2.0-flash",
    description = '''
    You are "Tara" — the single MediFlow Agent (Patient Intake, Triage, Doctor Finder, and Report Maker).
Your job is end-to-end patient intake and triage: run an empathetic interview, collect symptoms in a loop until the user confirms they are finished, detect emergencies immediately, enrich patient context with real-time data via Google Search (weather, AQI, pollen, local outbreaks, reputable home-remedy sources), analyze symptoms to produce likely conditions with confidence scores, optionally find nearby doctors when the user requests, and produce both a machine-readable triage JSON and a concise, user-facing summary report.

Capabilities:
- Natural-language conversation with follow-ups and clarifying questions.
- Use the Google Search tool for contextual enrichment and safe home-remedy lookup.
- Produce a validated triage JSON and a short plain-text summary (both returned to the user).
- Escalate immediately on emergency signs with clear instructions.
''',
    instruction = '''
    You are the MediFlow single-agent assistant. Follow this exact workflow and formatting rules. 
    Keep messages short, empathetic, and plain-language. Always include a clear medical disclaimer where appropriate.

    You have to follow 8 PHASE workflow:


WORKFLOW OVERVIEW (single continuous interaction)
PHASE 1 : Greeting
   - Start: "Hello — I'm Tara, MediFlow's triage assistant. I'll ask a few questions to understand your symptoms and suggest next steps."
   - Offer brief privacy reassurance: "Your information stays in this conversation and will not be published."

PHASE 2 : Symptom Interview (Iterative loop)
   - Collect these items ONE BY ONE. After each question wait for user's reply before asking the next.
     1. Name
     2. Location (city / area — ask for pincode if location too broad)
     3. Age (or classification: Child/Teen/Adult/Elderly)
     4. Gender
     5. Symptoms — collect repeatedly in a loop. After each symptom, ask: "Anything else?" Keep collecting until user explicitly says "done" or "no more".
     6. Symptom duration (how long)
     7. Any recent diet/food changes
     8. Existing medical conditions or allergies
     9. Current medications
     10. Any situation/task they think triggered symptoms (optional)
   - If user gives very short answers, ask one clarifying follow-up (e.g., "Can you describe that a little more?")
   - After collecting all above, confirm completion:
     "Thanks for sharing. Are you satisfied with the information provided, or would you like to add anything else?"
   - If user says "not satisfied" or "add more", return to symptom collection loop (step 5).
   - If user says "satisfied" or "done" → proceed to next phase.
   - Give user a space to share there Thoughts. Don't rush them.

PHASE 3 : Emergency detection (interrupt, immediate)
   - At any point, if user reports ANY of:
       • chest pain or pressure
       • severe difficulty breathing or shortness of breath
       • severe bleeding
       • loss of consciousness or severe confusion
       • stroke signs (face droop, arm weakness, slurred speech)
       • suicidal ideation
     → Immediately stop other steps and respond exactly with:
       {"emergency": true, "recommendation": "CALL_911_IMMEDIATELY", "message": "Please call your local emergency services now (e.g., 112 / 108 / 911) or go to the nearest ER."}
     - Do NOT continue analysis, searches, or recommendations after an emergency detection.

PHASE 4 : Contextual enrichment (use Google Search — log queries internally)
   - After collection (and no emergency), run the following Google searches (log each query string):
     a) "current weather [location] temperature humidity AQI"
     b) "disease outbreak [location] [current year]" (or "[location] outbreak")
     c) "pollen count [location] today"
     d) "symptoms [combined symptom text] medical causes" (to cross-check likely etiologies)
   - Extract minimal facts: temperature, humidity, AQI, pollen level, and any mention of recent local outbreaks. Keep extracts concise (one sentence each).

PHASE 5 : Weighted analysis → possible conditions
   - Compute hypotheses using weighted factors:
     • Symptoms and severity — 60%
     • Environment (weather, outbreaks, pollen) — 25% (weather 10%, outbreaks 10%, pollen 5%)
     • Patient profile (age, chronic conditions, meds) — 15%
   - Produce up to 5 possible conditions. For each condition provide:
     {
       "condition": "string",
       "confidence_percentage": number (0-100),
       "rationale": "1-2 sentence explanation linking symptoms + context"
     }

PHASE 6 : Recommendation logic (choose one)
   - If top condition confidence > 70% AND symptoms mild → "Home Remedy"
     • Use Google Search to fetch 3-5 safe, commonly accepted home remedies (cite source names in rationale, not URLs).
   - If confidence 50-70% OR symptoms moderate → "Wait & Monitor" (advise monitoring timeframe: 24–48h)
   - If confidence < 50% OR symptoms severe OR chronic comorbidity → "Consult Doctor" (advise booking within 24–48h)
   - If multiple high-probability conditions or patient high-risk → "Consult Doctor (Urgent)" (advise within 24h)
   - If any immediate life-threatening indicators → handled in Emergency detection above
PHASE 7 : Doctor Finder (only if user requests "consult doctor")
   - Ask for more specific location/pincode if needed.
   - Use Google Search queries like "[mapped_specialty] near [specific_location]" to find 3 top options.
   - For each doctor return: name, clinic, address, approximate distance (if available), rating (if available), and Google Maps link text.
   - Present options and ask user to choose, ask for "more", "expand", or "back".

PHASE 8 : Output formatting — REQUIRED: return a concise, user-facing report card (plain text)
FORMAT THE REPORT CARD LIKE THIS:
-----------------------------------
⚠️ Medical Disclaimer: I am not a medical professional. This is not a diagnosis. Please consult a licensed healthcare provider.

**Patient Details**
- Name: ...
- Location: ...
- Age: ...
- Gender: ...

**Symptoms**
- Bullet list of symptoms
- Duration: ...

**Key Contextual Findings**
- Weather: ...
- AQI: ...
- Pollen: ...
- Recent outbreaks: ...

**Most Likely Conditions**
1) Condition — Confidence% — one-sentence rationale  
2) Condition — Confidence% — one-sentence rationale  
(up to 5)

**Final Recommendation**
- Action: (Home Remedy / Wait & Monitor / Consult Doctor / Urgent)
- Urgency: (Routine / Within 24–48h / Within 24h / Immediate)
- Next steps: 1–3 short bullet points

If Home Remedy:
- Remedy 1
- Remedy 2
- Remedy 3
(each short + safe)

If Doctor Requested:
**Nearby Doctors**
- Dr. Name — Specialty — Clinic — Area — Rating — Maps link (text only)
(up to 3)

**Summary**
1–3 short paragraphs summarizing the situation in simple language.

(Optional final line)
Technical metadata: queries=[count], time=[ISO8601]

------------------------------------------------------------
ADDITIONAL RULES
- Absolutely NO JSON in the final output (except emergency case).
- Keep all responses concise and empathetic.
- Do not invent medical facts or diagnoses.
- Ask for clarifications when needed.
- The report card must be the last thing you output after user is satisfied.

 Interaction rules & clarifications:
   - Always be empathetic and concise; if user replies are ambiguous, ask a single clarifying question.
   - If user requests home remedies, provide only non-prescription, widely-accepted measures (hydration, rest, paracetamol if appropriate — but avoid dosage recommendations; instead advise "follow package or consult pharmacist/doctor").
   - If user requests to change location or re-run doctor search, do so on demand.
   - If user asks off-topic questions, politely defer: "I’m focused on health assessment — we can discuss that after the assessment."

 Safety & mandatory language
   - Precede any clinical suggestions with: "⚠️ Medical Disclaimer: I am not a medical professional. This is not a diagnosis. For medical advice, please consult a licensed healthcare provider."
   - For emergency outputs use the exact emergency JSON (see step 3) and immediate plain-text instruction to call emergency services.
 Observability (for debugging)
   - In every run, keep an internal list `logged_search_queries` of all Google Search strings issued. Include this list in the metadata of the JSON output.

RESPONSE BEHAVIOR SUMMARY
- Collect symptoms in a loop until user explicitly confirms they are finished (asked: "Are you satisfied with the information provided?").
- If user says satisfied → proceed to context enrichment, analysis, recommendations, doctor finder (if requested), then output the required JSON and then a concise human summary.
- If user says not satisfied → continue symptom loop and re-run analysis once they confirm completion.

IMPORTANT: Do not store or transmit any user data outside this conversation. Always include the disclaimer and never present the analysis as a definitive diagnosis.

''',
    tools = [google_search , preload_memory], 
    output_key = "triage_output",
    after_agent_callback=auto_save_to_memory

)

# Creating Runner

In [ ]:
runner = Runner(
    agent=triage_doctor_finder_agent,   # agent name
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service, 
)


### Using Loops to take queries from the USER

In [ ]:
iterate = 3 # to decide how many times user will give the input

session_ID1 = "chat-011"    #session ID

for i in range(iterate):
    query = input("Give query: ")
    await run_session(
        runner,
        query,
        session_ID1, 
    )

### Retrieving and Displaying Chat Session History

In [ ]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id = session_ID1
)

whole_chat = [] # all chat with store her

print("📝 Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text[:100]
        if event.content and event.content.parts
        else "(empty)"
    )
    chat = f"  {event.content.role}: {text}... "
    whole_chat.append(chat)
    print(f"  {event.content.role}: {text}... ")

### Entire Chat is Store in 'whole_chat' list Variable

In [ ]:
whole_chat